# Identifier les textes évoquant le concept de république

In [1]:
import pandas as pd
import re
import csv

df = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(215127, 59)

## INTRODUIRE PRÉ-TRAITEMENT TEXTE

Pour simplifier la vie et faciliter aussi possibles perf d'un futur modèle, virer les parenthèses et balises ici pour s'économiser pas mal de choses du côté des noms de groupes

In [2]:
# nettoyage moche du texte


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)
    # # TODO: aviser si enlève crochets, ou juste […] [...], surtout des troncatures citations
    # texte = re.sub(r"\[[^\]]*\]", "", texte)
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

## Regex

Logique de la tentative :
- regex
- mais exclure certains termes
- mais comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (les idées républicaines sont menacées par Les Républicains)

In [3]:
# préparer les pays à exclure
with open("../data/interim/liste_pays_republique.txt", "r", encoding="utf-8") as f:
    liste_pays = [line.strip() for line in f]

# créer un pattern regex pour les pays
# ici pas besoin d'avoir un groupe de capture pas pays mais juste global ok
pattern_pays = r"(\b(?:" + r"|".join(re.escape(p) for p in liste_pays) + r")\b)"


In [4]:
# Regex du champ lexical République (simplifié ici)

pattern_lexical = re.compile(
    r"républi", # même au milieu des mots
    re.I,
)

# Regex des expressions à exclure

# Expressions à exclure - casse exacte
pattern_excl_case_sensitive = re.compile(r"\b[LlDd]es Républicains\b") # voir pour élu Républicain ? doute

# Expressions à exclure - ignorer la casse
pattern_excl_case_insensitive = re.compile(
    r"|(\bprésident[s]? de la République\b)"
    r"|(\bprésidence[s]? de la République\b)"
    r"|(\bgauche démocrate et républicaine)"
    r"|(\brépublique en marche\b)"
    r"|(\bprocureur[s]? de la République\b)"
    r"|(\bcour[s]? de justice de la République\b)"
    r"|(\badministration générale de la République\b)"
    r"|(" + pattern_pays + ")",  # ajout des exclusions de pays si existe
    re.I,
)

def contains_lexical_outside_excl(text):
    # Trouver les positions des expressions exclues
    excl_positions = []

    # Ajouter les exclusions sensibles à la casse
    excl_positions.extend(
        [m.span() for m in pattern_excl_case_sensitive.finditer(text)]
    )

    # Ajouter les exclusions insensibles à la casse
    excl_positions.extend(
        [m.span() for m in pattern_excl_case_insensitive.finditer(text)]
    )

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False

In [5]:
# # bloc d'essai
# mon_texte = "La république démocratique du congo et instituteurs de la République."
# contains_lexical_outside_excl(mon_texte)


In [6]:
# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [7]:
df_match = df[df["repu_match_valide"]]
df_match.shape

(3668, 61)

In [8]:
df_match.to_csv(
    "../data/interim/df_repu.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [9]:
# vérif écriture/lecture ok
df_test = pd.read_csv("../data/interim/df_repu.csv")
df_test.shape

(3668, 61)

In [10]:
pattern_excl_case_insensitive

re.compile(r"|(\bprésident[s]? de la République\b)|(\bprésidence[s]? de la République\b)|(\bgauche démocrate et républicaine)|(\brépublique en marche\b)|(\bprocureur[s]? de la République\b)|(\bcour[s]? de justice de la République\b)|(\badministration générale de la République\b)|((\b(?:République\ islamique\ d'Afghanistan|République\ d'Afrique\ du\ Sud|République\ d'Albanie|République\ algérienne\ démocratique\ et\ populaire|République\ fédérale\ d'Allemagne|République\ d'Angola|République\ argentine|République\ d'Arménie|République\ d'Autriche|République\ d'Azerbaïdjan|République\ populaire\ du\ Bangladesh|République\ du\ Bénin|République\ de\ Biélorussie|République\ de\ Bolivie|République\ du\ Botswana|République\ fédérative\ du\ Brésil|République\ de\ Bulgarie|République\ du\ Burundi|République\ du\ Cameroun|République\ du\ Cap\-Vert|République\ centrafricaine|République\ du\ Chili|République\ populaire\ de\ Chine|République\ de\ Chypre|République\ de\ Colombie|Congo\ \(la\ Républiq